# Cantilever subjected to end moment with HHJ formulation

In this notebook we consider a cantilever, which is fixed on the left and subjected to an end moment at the right, such that it should form a perfect circle. Symmetry boundary conditions are used at the top and bottom boundary to avoid bifurcations.

The material and geometrical properties are
$$
E = 1.2\times 10^6,\qquad \nu = 0,\qquad L=12,\qquad W=1, \qquad t=0.1,
$$
and the maximal moment is $M_{\max}=50\pi/3$.

<center><img src="pictures/cant_bend_mom_1d.png" width="200"/></center>

We consider a $16\times 1$ quadrilateral grid with clamped boundary conditions on the left, apply a bending moment at the right, and the other boundaries are left free. 

We use the two-field hybridized HHJ formulation for nonlinear Koiter shells: Find the displacement field $u$, bending moment tensor $\boldsymbol{m}$, and hybridization field $\alpha$, $(u,\boldsymbol{m},\alpha)\in U_h^k\times M_h^{k-1}\times V_h^{k-1}$, for the Lagrangian 
\begin{align*}
\mathcal{L}(u&,\boldsymbol{m},\alpha) =\frac{t}{2}\|\boldsymbol{E}(u)\|^2_{\mathbb{M}} -\frac{6}{t^3}\|\boldsymbol{m}\|^2_{\mathbb{M}^{-1}} +  \sum_{T\in\mathcal{T}_h}\int_{T} \frac{1}{J}\boldsymbol{m}:(\boldsymbol{H}_{\nu\circ\phi}+(J-\hat{\nu}\cdot\nu\circ\phi)\nabla_S\hat{\nu})\,dx \\
&\qquad+ \sum_{E\in\mathcal{E}_h}\int_E\big(\sphericalangle(\hat{\nu}_L,\hat{\nu}_R)-\sphericalangle(\nu_L,\nu_R)\circ\phi\big)\{\!\{\boldsymbol{m}_{\hat{\mu}\hat{\mu}}\}\!\}+\alpha_{\hat{\mu}_E}[\![\boldsymbol{m}_{\hat{\mu}\hat{\mu}}]\!]\,ds - \int_{\Gamma_{\mathrm{right}}}P_{\max}\,u_z\,ds,
\end{align*}
where $\boldsymbol{H}_{\nu\circ\phi}=\sum_{i=1}^3(\nabla_S^2u_i)\nu_i\circ\phi$, and $\nabla^2_S u_i=\boldsymbol{P}\nabla_S(\nabla_S u_i)$ denotes the Riemann surface Hessian. Further, $\boldsymbol{E}=\frac{1}{2}(\boldsymbol{F}^\top\boldsymbol{F}-\boldsymbol{P})= \frac{1}{2}(\nabla_S u^\top \nabla_S u + \nabla_S u^\top\boldsymbol{P} + \boldsymbol{P}\nabla_S u)$ denotes the Green-strain tensor restricted on the tangent space measuring the membrane energy of the shell, $t$ the shell thickness, and $\mathbb{M}$ the material tensor. $\nu$ and $\hat{\nu}$ are the unit normal vectors with respect to the deformed and initial configuration, respectively. $\hat{\tau}$ and $\hat{\mu}$ are the edge-tangential and co-normal (element-normal) vectors on the initial configuration, respectively.
    
<center><img src="pictures/nv_conv_tang_trig.png" width="200"> </center>

Further, $J=\|\mathrm{cof}(\boldsymbol{F})\hat{\nu}\|$ is the surface measure.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
import netgen.meshing as meshing
import ipywidgets as widgets

# Geometry and material parameters
thickness = 0.1
L = 12
W = 1
E, nu = 1.2e6, 0
# Maximal moment applied at the right edge
moment = IfPos(x - L + 1e-6, 1, 0) * 50 * pi / 3

# Create geometry and mesh
geom = meshing.SurfaceGeometry(lambda x, y, z: (L * x, W * y, 0))
mesh = Mesh(geom.GenerateMesh(quads=True, nx=16, ny=1))
Draw(mesh);

At the clamped boundary we need to fix the displacement by homogeneous Dirichlet boundary conditions and the hybridization variable, which has the physical meaning of the rotated angle.

In [ ]:
# Set polynomial order for displacement field u, the bending moment
# tensor m and hybrid variable hyb have one degree less
order = 2

# Create finite element spaces
# Clamped at left edge (bbnd ... co-dimension 2 boundary of 3D mesh)
# and symmetry boundary condition at the top and bottom edges
fesU = VectorH1(
    mesh,
    order=order,
    dirichletx_bbnd="left",
    dirichlety_bbnd="left|top|bottom",
    dirichletz_bbnd="left",
)
fesM = HDivDivSurface(mesh, order=order - 1, discontinuous=True)
fesH = NormalFacetSurface(mesh, order=order - 1, dirichlet_bbnd="left|top|bottom")

fes = fesU * fesM * fesH

# Symbolic trial functions for displacement field u,
# bending moment tensor m and hybrid variable hyb
u, m, hyb = fes.TrialFunction()
# Trace needed for m and hyb as we are on the surface
m, hyb = m.Trace(), hyb.Trace()

# Regge FESpace to avoid membrane locking
Regge = HCurlCurl(mesh, order=order - 1, discontinuous=True)

# GridFunction to store the solution
gf_solution = GridFunction(fes, name="solution")

We define the unit normal vector $\hat\nu$, edge-tangential vector $\hat t$ and the co-normal vector $\hat\mu = \hat\nu\times \hat t$ at the initial configuration.

Then the projection operator onto the tangent space, deformation gradient, Cauchy-Green, and Green tensors $\boldsymbol{P}$, $\boldsymbol{F}$, $\boldsymbol{C}$, and $\boldsymbol{E}$ are introduced.

Finally, the unit normal, edge-tangential, and co-normal vectors $\nu, \tau,\mu$ on the deformed configuration are declared, which depend on the unknown displacement field $u$.

In [ ]:
# Surface unit normal, edge-tangential, and co-normal vectors on initial configuration
nv = specialcf.normal(mesh.dim)
tv = specialcf.tangential(mesh.dim)
cnv = Cross(nv, tv)

# Projection to the surface tangent space
Ptau = Id(mesh.dim) - OuterProduct(nv, nv)
# Surface deformation gradient
Ftau = Grad(u).Trace() + Ptau
# Surface Cauchy-Green tensor
Ctau = Ftau.trans * Ftau
# Surface Green-Lagrange strain tensor
Etautau = 0.5 * (Ctau - Ptau)
# surface determinant
J = Norm(Cof(Ftau) * nv)

# Surface unit normal, edge-tangential, and co-normal vectors on deformed configuration
nv_def = Normalize(Cof(Ftau) * nv)
tv_def = Normalize(Ftau * tv)
cnv_def = Cross(nv_def, tv_def)

# Surface Hessian weighted with unit normal vector on deformed configuration
H_nv_def = (u.Operator("hesseboundary").trans * nv_def).Reshape((3, 3))

For the angle computation of the bending energy we use an averaged unit normal vector avoiding the necessity of using information of two neighbored element at once (+ a more stable formulation using the co-normal vector instead of the unit normal vector)

<center><img src="pictures/nonsmooth_av_nv_el_nv.png" width="150"> </center>

\begin{align*}
\sum_{E\in\mathcal{E}_h}\int_E(\sphericalangle(\hat{\nu}_L,\hat{\nu}_R)-\sphericalangle(\nu_L,\nu_R)\circ\phi)\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,ds &= \sum_{T\in\mathcal{T}_h}\int_{\partial T}(\sphericalangle(\mu\circ\phi,P^{\perp}_{\tau\circ\phi}(\{\!\{\nu^n\}\!\}))-\sphericalangle(\hat{\mu},\{\!\{\hat{\nu}\}\!\}))\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,ds,
\end{align*}
where 
$$
P^{\perp}_{\tau\circ\phi}(v)= \frac{1}{\|\boldsymbol{P}^{\perp}_{\tau\circ\phi}v\|}\boldsymbol{P}^{\perp}_{\tau\circ\phi}v,\qquad \boldsymbol{P}^{\perp}_{\tau\circ\phi}= \boldsymbol{I}-\tau\circ\phi\otimes\tau\circ\phi
$$
denotes the (nonlinear) normalized projection to the plane perpendicular to the deformed edge-tangential vector $\tau$ for measuring the correct angle.

In [ ]:
# Save clamped and symmetry boundary for updating
# averaged unit normal vector during load-steps
gf_clamped_bnd = GridFunction(FacetSurface(mesh, order=0))
gf_clamped_bnd.Set(1, definedon=mesh.BBoundaries("left"))
gf_sym_bnd = GridFunction(gf_clamped_bnd.space)
gf_sym_bnd.Set(1, definedon=mesh.BBoundaries("top|bottom"))

# Unit normal vector on current configuration
# Used to update averaged unit normal vector during load-steps
cf_nv_cur = Normalize(Cof(Ptau + Grad(gf_solution.components[0])) * nv)

# FE space for averaged normal vectors living only on the edges of the mesh
fesVF = VectorFacetSurface(mesh, order=order - 1)
# GridFunctions to save averaged normal vectors on deformed and initial configurations
averaged_nv = GridFunction(fesVF)
averaged_nv_init = GridFunction(fesVF)

# Initialize by averaging unit normal vectors on initial configuration
# definedon=mesh.Boundaries(".*") is needed as we interpolate on the surface mesh
averaged_nv.Set(nv, dual=True, definedon=mesh.Boundaries(".*"))
averaged_nv_init.Set(nv, dual=True, definedon=mesh.Boundaries(".*"))
# Normalize averaged normal vector on initial configuration
cf_averaged_nv_init_norm = Normalize(averaged_nv_init)
# Project averaged unit normal vector being perpendicular to deformed edge-tangent vector
# to measure correct angle on deformed configuration
cf_proj_averaged_nv = Normalize(averaged_nv - (tv_def * averaged_nv) * tv_def)

Define the material and inverse material norms $\|\cdot\|_{\mathbb{M}}^2$ and $\|\cdot\|_{\mathbb{M}^{-1}}^2$ with Young modulus $\bar{E}$ and Poisson's ratio $\bar{\nu}$
\begin{align*}
\mathbb{M} \boldsymbol{E} = \frac{\bar E}{1-\bar \nu^2}\big((1-\bar \nu)\boldsymbol{E}+\bar \nu\,\mathrm{tr}(\boldsymbol{E})\boldsymbol{P}\big),\qquad\mathbb{M}^{-1} \boldsymbol{m} = \frac{1+\bar \nu}{\bar E}\big(\boldsymbol{m}-\frac{\bar \nu}{\bar\nu+1}\,\mathrm{tr}(\boldsymbol{m})\boldsymbol{P}\big).
\end{align*}

In [ ]:
# Material norm
def MaterialNorm(mat, E, nu):
    return E / (1 - nu**2) * ((1 - nu) * InnerProduct(mat, mat) + nu * Trace(mat) ** 2)


# Material stress
def MaterialStress(mat, E, nu):
    return E / (1 - nu**2) * ((1 - nu) * mat + nu * Trace(mat) * Ptau)


# Inverse of the material norm
def MaterialNormInv(mat, E, nu):
    return (1 + nu) / E * (InnerProduct(mat, mat) - nu / (nu + 1) * Trace(mat) ** 2)

Define shell energies, where the bending energy is incorporated as a saddle point problem. We set ``condense=True`` in the bilinear form to compute the Schur complement eliminating the bending moment tensor unknowns $\boldsymbol{m}$ from the global system. Thus, we obtain a minimization problem in $(u,\alpha)$.

In [ ]:
# Bilinear form for problem
# We define the Lagrangian of the HHJ formulation. Therefore,
# we use Variation() such that Newton knows to build the first variation
# for the residual and the second variation for the stiffness matrix. The
# stiffness matrix  will be symmetric and we use static condensation via
# condense = True to eliminate the bending moment tensor m from the global system.
# We use .Compile() to simplify (linearize) the coefficient expression tree.

bfA = BilinearForm(fes, symmetric=True, condense=True)
# Membrane energy
# Interpolate the membrane strains Etautau into the Regge space
# to avoid membrane locking
bfA += Variation(
    0.5 * thickness * MaterialNorm(Interpolate(Etautau, Regge), E, nu) * ds
).Compile()
# Bending energy
# Element terms of bending energy
bfA += Variation(
    (
        -6 / thickness**3 * MaterialNormInv(m, E, nu)
        + InnerProduct(H_nv_def + (J - nv * nv_def) * Grad(nv), 1 / J * m)
    )
    * ds
).Compile()
# Boundary terms of bending energy including hybridization variable
bfA += Variation(
    (
        acos(cnv_def * cf_proj_averaged_nv)
        - acos(cnv * cf_averaged_nv_init_norm)
        + hyb * cnv
    )
    * (m * cnv * cnv)
    * ds(element_boundary=True)
).Compile()

# moment at the right boundary as natural boundary condition for the
# hybridization unknown. Parameter par for load-stepping below.
par = Parameter(0.0)
bfA += Variation(-par * moment * (hyb * cnv) * ds(element_boundary=True))

In [ ]:
# Reset the solution to zero
gf_solution.vec[:] = 0
# Extract components of solution
gfu, gfm, _ = gf_solution.components

# Draw displacement, bending moment tensor, and membrane stresses
scene = Draw(gfu, mesh, "displacement", deformation=gfu)
scene2 = Draw(Norm(gfm), mesh, "bending_stress", deformation=gfu)


# Exact displacement solution
R_ex = 100 / (par * 50 * pi / 3)
ex_deformation = CF((R_ex * sin(x / R_ex) - x, 0, -R_ex * cos(x / R_ex) + R_ex))

# Exact unit normal vector on deformed configuration
ex_grad = CF(
    (ex_deformation.Diff(x), ex_deformation.Diff(y), ex_deformation.Diff(z)),
    dims=(3, 3),
).trans
ex_nv = Normalize(Cof(Ptau + ex_grad) * nv)

# Exact Hessian weighted with unit normal vector on deformed configuration
ex_hesse_1 = (
    Ptau
    * CF(
        (ex_grad[0, :].Diff(x), ex_grad[0, :].Diff(y), ex_grad[0, :].Diff(z)),
        dims=(3, 3),
    ).trans
)
ex_hesse_2 = (
    Ptau
    * CF(
        (ex_grad[1, :].Diff(x), ex_grad[1, :].Diff(y), ex_grad[1, :].Diff(z)),
        dims=(3, 3),
    ).trans
)
ex_hesse_3 = (
    Ptau
    * CF(
        (ex_grad[2, :].Diff(x), ex_grad[2, :].Diff(y), ex_grad[2, :].Diff(z)),
        dims=(3, 3),
    ).trans
)
Hn_ex = ex_hesse_1 * ex_nv[0] + ex_hesse_2 * ex_nv[1] + ex_hesse_3 * ex_nv[2]

# Exact bending moment tensor
bend_ex = thickness**3 / 12 * MaterialStress(Hn_ex, E, nu)

Use Newton's method for solving and increase magnitude of right-hand side by load-steps.

The unit normal vector is averaged after each load-step.

An exact solution is known to this problem.

In [ ]:
# Define point of interest and store results
point_P = (L, W / 2, 0)
result_P = [(0, 0, 0)]
ex_sol = [(0, 0, 0)]


def GetExactSolution(par):
    R = 100 / (par.Get() * 50 * pi / 3)
    return (R * sin(L / R) - L, 0, -R * cos(L / R) + R)


# Use num_steps uniform load-steps
num_steps = 20

tw = widgets.Text(value="step = 0")
display(tw)

# Thread parallel
with TaskManager():
    for steps in range(num_steps):
        par.Set((steps + 1) / num_steps)
        tw.value = f"step = {steps+1}/{num_steps}"

        # Update averaged normal vector
        # On clamped boundary it remains the unit normal vector of the initial configuration
        # On symmetry boundary the co-normal direction of the unit normal vector is constraint
        # On the rest of the boundary it is updated by the current unit normal vector
        averaged_nv.Set(
            (1 - gf_clamped_bnd - gf_sym_bnd) * cf_nv_cur
            + gf_sym_bnd * (cf_nv_cur - (cf_nv_cur * cnv) * cnv)
            + gf_clamped_bnd * nv,
            definedon=mesh.Boundaries(".*"),
            dual=True,
        )

        # Use Newton solver with residual tolerance 1e-5 and maximal 100 iterations
        # Due to hybridization techniques we can use the sparsecholesky solver for
        # solving the Schur complement (done internally)
        solvers.Newton(
            bfA,
            gf_solution,
            inverse="sparsecholesky",
            printing=False,
            maxerr=1e-5,
            maxit=100,
        )
        # Redraw solutions
        scene.Redraw()
        scene2.Redraw()

        result_P.append((gfu(mesh(*point_P, BND))))
        ex_sol.append(GetExactSolution(par))

In [ ]:
print(
    "Relative error of bending moment tensor = ",
    sqrt(Integrate(InnerProduct(bend_ex - gfm, bend_ex - gfm), mesh, BND))
    / sqrt(Integrate(InnerProduct(bend_ex, bend_ex), mesh, BND)),
)

Plot the solution and compare with exact values.

In [ ]:
import matplotlib.pyplot as plt

u_x, _, u_z = zip(*result_P)
y_axis = [i / num_steps for i in range(len(u_x))]
u_x = [-val for val in u_x]
u_ex_x, _, u_ex_z = zip(*ex_sol)
u_ex_x = [-val for val in u_ex_x]

plt.plot(u_x, y_axis, "-*", label="$-u_x$")
plt.plot(u_z, y_axis, "-x", label="$u_z$")
plt.plot(u_ex_x, y_axis, "--", color="k", label="$-u\\_ex_x$")
plt.plot(u_ex_z, y_axis, "-.", color="k", label="$u\\_ex_z$")

plt.xlabel("displacement")
plt.ylabel("$M/M_{\\max}$")
plt.legend()
plt.show()